## 接口

- server.py文件

- tuling.py文件

In [ ]:
# server.py 主文件
import eventlet

eventlet.monkey_patch()

import socketio
import eventlet.wsgi
from tuling import get_tuling

# 创建socketIO服务器，推荐使用协程的方式

# 步骤：
# 1、需要使用协程的模块，eventlet,
# 2、导入socketIO模块
# 3、创建socketIO服务对象，需要指定以协程的方式运行
# 4、创建服务器实例对象
# 5、监听主机和端口
# 6、启动服务器

sio = socketio.Server(async_mode='eventlet')
app = socketio.Middleware(sio)
# 5、监听主机和端口
sock = eventlet.listen(('', 8000))


# 提供聊天服务的功能,默认回复hello world
# @sio.on('message')
# def on_message(sid,data):
#     print('sid={},data={}'.format(sid,data))
#
#     temp_msg = {
#         'msg':'hello world'
#     }
#     sio.emit('message',temp_msg,room=sid)

# 提供聊天服务的功能,使用聊天机器人进行回复
@sio.on('message')
def on_message(sid, data):
    print('sid={},data={}'.format(sid, data))

    # 要求客户端传输的数据的格式
    user_msg = data.get('msg', '')
    # 如果用户的数据格式满足条件，调用图灵机器人
    if user_msg:
        if user_msg == '你好':
            msg = '你也好！！！'
        # 如果用户不是说你好，调用图灵机器人，获取真正的回复内容
        else:
            msg = get_tuling(user_msg, sid)

        sio.emit('message', msg, room=sid)


# 6、启动服务器
eventlet.wsgi.server(sock, app)


In [ ]:
# tuling.py 文件
import requests
import json

# 步骤：
# 1、导入发送网络请求的模块，requests(需要安装)、urllib
# 2、构造请求信息
# 3、发送请求，获取图灵机器人回复的响应数据
# 4、解析响应数据，返回给socketIO服务器

# 图灵机器人的地址
url = 'http://openapi.tuling123.com/openapi/api/v2'


# 定义函数，给socketIO服务器进行调用
def get_tuling(text, sid):
    # 使用字典定义请求信息
    req_dict = {
        "perception": {
            "inputText": {
                "text": text
            },
        },
        "userInfo": {
            "apiKey": "816bbb346d8e4f41b5fe91dc00bd2ca2",
            "userId": sid
        }
    }
    # 指定请求头信息的类型为json
    header = {
        'Content-Type': 'application/json'
    }
    # 使用requests模块，向图灵机器人接口发送post请求
    resp = requests.post(url, data=json.dumps(req_dict), headers=header)
    # 把图灵机器人返回的响应数据进行解码
    content = resp.content.decode()
    # 把响应的数据，转成字典
    resp_dict = json.loads(content)
    results = resp_dict['results']
    # 遍历列表
    for result in results:
        if result['resultType'] != 'text':
            continue
        else:
            text = result['values']['text']
    print('图灵机器人回复={}'.format(text))
    return text
